# Computing thermal conductivity of a rotor chain via NEMD

Objective: simulate a one-dimensional chain of classical rotors , with interaction potential
$$V(r) = 1 - \cos(r)$$
oupled to boundary heat baths and subject to thermal forcing to estimate its thermal conductivity using the NEMD (Nonequilibrium Molecular Dynamics) approach. Rotor chains have finite thermal conductivity (see Giardinà _et al_ (2000), Gendelman and Savin (2000,2005), Yang and Hu (2005), Das and Dhar (2014).} $\kappa_N$ dramatically decreasing for $\bar{T} > 0.5$ (Giardin\'a {\it et al.} (2000))

## Parameters:

- chain length $10 \le n \le 100$
- no pinning ($U=0$)
- boundary temperatures: $T_L = T_{\mathrm{eq}} - \Delta T / 2$, $T_R = T_{\mathrm{eq}} + \Delta T / 2$ with $T_{\mathrm{eq}} < 0.5$
- boundary conditions: fixed on the left and free on the right
- friction parameters: $\xi_L=1$, $\xi_R=1$
- thermalization time steps `Nt_thrm`
- simulation time steps `Nt_sim`
- time step `dt` $\in[0.001,0.5]$ (depending on the chosen integration scheme)

## Dynamics:
Recall the equations: for all $i \in [[1,n]]$
$$
dq_i = p_i \, dt
$$
$$
dp_i = - \Big( V'(q_i-q_{i-1}) - V'(q_{i+1}-q_i)\Big) dt
+ \delta_{i,1} \Big( -\xi p_1 \, dt + \sqrt{2\xi T_L} \, dW_{1,t}\Big)
+ \delta_{i,n} \Big( -\xi p_n \, dt + \sqrt{2\xi T_R} \, dW_{n,t}\Big)
$$
Integration schemes: BABO, OBABO, ...

## Steps

- Initialize the system: Ensure the initial positions and momenta are appropriately set. This includes thermalizing with the correct initial conditions.

- Run the dynamics: Integrate the equations of motion using the numerical scheme provided.

- Make sure the system reaches a stationary state: check for local equilibrium by plotting the histogram of the marginals in $q$ and $p$ 

- Compute the energy current, using $j_{i,i+1} =-\dfrac 1 2 (p_i + p_{i+1}) V'(q_{i+1}-q_i)$

- Compute $$ \widehat\kappa_n^{N_\text{Niter}} = \frac 1{\Delta T} \frac 1{N_\text{Niter}}
		\sum_{m=1}^{N_\text{Niter}} J_n(q^m, p^m)$$

Optional

- Plot the temperature profile in the chain $T_i = \mathbb{E}_{\mu_{\Delta T, \Delta t}}[p_i^2]$, $i \in [[1,n]]$.
- Plot the instantaneous total current

In [ ]:
# using Pkg; 
# Pkg.add("Random")
using Random
using Statistics
using Distributions
using SpecialFunctions, NumericalIntegration # or QuadGK
using Plots

In [ ]:
#  global parameters
const n = 50     # number of atoms
const BCl = "fixed"
const BCr = "free"
const Nt_thrm = 100000  # number of thermalization time steps
const Nt_sim =  10000000   # number of simulation time steps
const dt = 0.001 # time step
const T_eq = 0.2   # reference equilibrium temperature
const ΔT = 0.05  # temperature difference at the boundaries
const ξ_L = 1.0
const ξ_R = 1.0
const storage_interval = 10 # save q's and p's for histograms every storage_interval iterations
const seed = 1234    # fix a random seed to ensure reproducibility 
const potential = "rotor"

In [ ]:
function initialize_T_profile() # uses global (T_eq, ΔT, n)
# compute a linear profile of temperature between TL and TR
    return init_T_profile
end

In [ ]:
function initialize_chain(imposed_T_profile) # uses global n
# initialize the q's qnd p's
# with q ∈ [-π,π]
    return q, p
end

In [ ]:
# Interaction potentials and forces

function interaction_potential(r)
.....
end
 
function interaction_force(r) 
.....
end

In [ ]:
# Check what this function does
function compute_dist_wt_BC(q) # uses global (n, BCl="free", BCr ="free")
    dist_vec_wt_BC = zeros(n+1)
    dist_vec_wt_BC[2:(end-1)] = q[2:end] - q[1:(end-1)]
    
    if BCl == "fixed"
        dist_vec_wt_BC[1] = q[1] # q_1-q_0 with q_0=0
    elseif BCl == "periodic"
        dist_vec_wt_BC[1] = q[1] - q[n] # q_1-q_0 with q_0=q_n
    end
    
    if BCr == "fixed"
        dist_vec_wt_BC[n] = -q[n] # q_(n+1)-q_n with q_(n+1) =0
    elseif BCr == "periodic"
        dist_vec_wt_BC[n] =  q[1] - q[n] # q_(n+1)-q_n with q_(n+1) = q_1
    end

    if potential == "rotor"
        dist_vec_wt_BC .= mod.(dist_vec_wt_BC .+ π, 2π) .- π
    end

    return dist_vec_wt_BC   # dim n+1
end

In [ ]:
function ham_force_field_per_site(q)
# compute the force field on each site
end

In [ ]:
function OU_step!(p, thrm_atoms::Vector{Int}, T_vec::Vector{Float64}, ξ_vec::Vector{Float64},a)
    mydt = dt*a
# write a function that can implement a OU step on every number of atom
end

In [ ]:
function step_verlet!(q, p, thrm_atoms::Vector{Int}, T_vec::Vector{Float64}, ξ_vec::Vector{Float64}, f)  # BABO
    p .+= 0.5dt .* f
    q .+= dt .* p
    if potential == "rotor"
        q .= mod.(q .+ π, 2π) .- π
    end
    
    f .= ham_force_field_per_site(q)
    p .+= 0.5dt .* f

    OU_step!(p, thrm_atoms, T_vec, ξ_vec)
end

In [ ]:
function step_obabo!(q, p, thrm_atoms::Vector{Int}, T_vec::Vector{Float64}, ξ_vec::Vector{Float64}, f)
    # implement the OBABO integration scheme
end

In [ ]:
function step_baoab!(q, p, thrm_atoms::Vector{Int}, T_vec::Vector{Float64}, ξ_vec::Vector{Float64}, f)
    # implement the BAOAB scheme
end

## If you correctly implemented the previous functions, `simulate()` is ready to use. 

Inspect the function, add explanatory comments and modify it as desired.

In [ ]:
function simulate(scheme) # uses global (n, dt, Nt_sim, T_eq, ΔT, potential="rotor")

    Random.seed!(seed)  # to be able to reproduce the results
    
    imposed_T_profile = initialize_T_profile()# (T_eq, ΔT, n)
    q, p = initialize_chain(imposed_T_profile)

    computed_T_profile = zeros(n)
    Jntot = zeros(Nt_sim)
    r_hist = [] # creates an empty Abstract array; I could do = Vector{Vector{Float64}}() since I want to store vectors of Float64
    p_hist = []

    f = ham_force_field_per_site(q)

    if scheme == "verlet"

        thrm_atoms = collect(1:n)    # thermalization phase
        ξ_vec = ones(length(thrm_atoms))
        for step in 1:Nt_thrm  
            step_verlet!(q, p, thrm_atoms, imposed_T_profile[thrm_atoms], ξ_vec, f)
        end
        
        thrm_atoms = [1,n]  # simulation phase
        ξ_vec = [ξ_L, ξ_R]
        for step in 1:Nt_sim
            step_verlet!(q, p, thrm_atoms, imposed_T_profile[thrm_atoms], ξ_vec, f)
            computed_T_profile .+= p .^ 2
            Jntot[step] = sum(0.5 .* (p[1:end-1] +p[2:end]) .* f[2:end])
            if step % storage_interval == 0
                r = compute_dist_wt_BC(q)[1:n]
                push!(r_hist, copy( [r[floor(Int,n/4)], r[floor(Int,n/2)], r[floor(Int,3*n/4)]] ))
                push!(p_hist, copy( [p[floor(Int,n/4)], p[floor(Int,n/2)], p[floor(Int,3*n/4)]] ))
            end
        end
        
    elseif scheme == "baoab"
        
        thrm_atoms = collect(1:n)    # thermalization phase
        ξ_vec = ones(length(thrm_atoms))
        for step in 1:Nt_thrm  
            step_baoab!(q, p, thrm_atoms, imposed_T_profile[thrm_atoms], ξ_vec, f)
        end
        
        thrm_atoms = [1,n]  # simulation phase
        ξ_vec = [ξ_L, ξ_R]
        for step in 1:Nt_sim
            step_baoab!(q, p, thrm_atoms, imposed_T_profile[thrm_atoms], ξ_vec, f)
            computed_T_profile .+= p .^ 2
            Jntot[step] = sum(0.5 .* (p[1:end-1] +p[2:end]) .* f[2:end])
            if step % storage_interval == 0
                r = compute_dist_wt_BC(q)[1:n]
                push!(r_hist, copy( [r[floor(Int,n/4)], r[floor(Int,n/2)], r[floor(Int,3*n/4)]] ))
                push!(p_hist, copy( [p[floor(Int,n/4)], p[floor(Int,n/2)], p[floor(Int,3*n/4)]] ))
            end
        end
        
    else
        error("Unknown integration scheme: $scheme")
    end

    # compute temperature profile
    computed_T_profile ./= Nt_sim

    return r_hist, p_hist, computed_T_profile, Jntot
end

In [ ]:
function NEMD_estimate_of_conductivity(Jntot) # (Jntot, ΔT)
# write the function that computes the thermal conductivity
end

In [ ]:
# ou can also change here the values of the parameters
# Nt_thrm = 100000  # number of thermalization time steps
# Nt_sim =  10000000   # number of simulation time steps
# dt = 0.001 # time step
println("Running Verlet scheme...")
r_hist, p_hist, computed_T_profile, Jntot = simulate("verlet")
if ΔT != 0.0
    κ_v = NEMD_estimate_of_conductivity(Jntot)
end
println("Average total current (Verlet): ", mean(Jntot))
println("Estimated conductivity (Verlet): ", κ_v)

## Sanity checks 
Functions to check local equilibrium, plot temperature profile and instantaneous current

In [ ]:
function plot_histograms(r_hist, p_hist, Ts)
    for idx in 1:length(r_hist[1]) # = length(phist(1))

        β = 1/Ts[idx]

        # POSITIONS

        # plot the position histogram        
        # plot for comparison the theoretical probability (ie normalized!) density function 

        # MOMENTA
        # plot the position histogram        
        # plot for comparison the theoretical probability density function

    end
end

In [ ]:
plot_histograms(r_hist, p_hist, computed_T_profile[[n ÷ 4, n ÷ 2, 3n ÷ 4]])

In [ ]:
# plot the temperature profile

In [2]:
# plot the instantaneous total current 

## Bonus: check energy conservation for Verlet and BAOAB scheme